# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [2]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow --user

# Installing Keras
! pip install --upgrade keras --user

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn --user

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\Scripts\pip-script.py", line 10, in <module>
    sys.exit(main())
TypeError: 'module' object is not callable
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\Scripts\pip-script.py", line 10, in <module>
    sys.exit(main())
TypeError: 'module' object is not callable
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\Scripts\pip-script.py", line 10, in <module>
    sys.exit(main())
TypeError: 'module' object is not callable
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\Scripts\pip-script.py", line 10, in <module>
    sys.exit(main())
TypeError: 'module' object is not callable


# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [3]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [7]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [8]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 93us/step - loss: 0.4880 - accuracy: 0.7955
Epoch 2/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.4276 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4212 - accuracy: 0.8142
Epoch 4/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4148 - accuracy: 0.8284
Epoch 5/100
8000/8000 [==============================] - 1s 75us/step - loss: 0.4106 - accuracy: 0.8316
Epoch 6/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4069 - accuracy: 0.8340
Epoch 7/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4047 - accuracy: 0.8335
Epoch 8/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4032 - accuracy: 0.8340
Epoch 9/100
8000/8000 [==============================] - 1s 81us/step - loss: 0.4023 - accuracy: 0.8353
Epoch 10/100
8000/8000 [==============================] - 1s 78u

Epoch 79/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.3354 - accuracy: 0.8664
Epoch 80/100
8000/8000 [==============================] - 1s 81us/step - loss: 0.3367 - accuracy: 0.8639
Epoch 81/100
8000/8000 [==============================] - 1s 82us/step - loss: 0.3360 - accuracy: 0.8619
Epoch 82/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.3360 - accuracy: 0.8633
Epoch 83/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.3356 - accuracy: 0.8635
Epoch 84/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.3361 - accuracy: 0.8636
Epoch 85/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.3359 - accuracy: 0.8643
Epoch 86/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.3356 - accuracy: 0.8636
Epoch 87/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.3355 - accuracy: 0.8650
Epoch 88/100
8000/8000 [==============================]

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [10]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1537   58]
 [ 202  203]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [12]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 100us/step - loss: 0.5085 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.4835 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4575 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4382 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4324 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.4310 - accuracy: 0.7956
Epoch 7/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4295 - accuracy: 0.7964
Epoch 8/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.4293 - accuracy: 0.7950
Epoch 9/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4288 - accuracy: 0.7952
Epoch 10/100
8000/8000 [==============================] - 1s 79

Epoch 79/100
8000/8000 [==============================] - 1s 81us/step - loss: 0.3597 - accuracy: 0.8522
Epoch 80/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.3608 - accuracy: 0.8516
Epoch 81/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.3593 - accuracy: 0.8524
Epoch 82/100
8000/8000 [==============================] - 1s 81us/step - loss: 0.3733 - accuracy: 0.8512
Epoch 83/100
8000/8000 [==============================] - 1s 82us/step - loss: 0.3649 - accuracy: 0.8510
Epoch 84/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.3573 - accuracy: 0.8516
Epoch 85/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.3577 - accuracy: 0.8515
Epoch 86/100
8000/8000 [==============================] - 1s 81us/step - loss: 0.3578 - accuracy: 0.8541
Epoch 87/100
8000/8000 [==============================] - 1s 80us/step - loss: 0.3559 - accuracy: 0.8525
Epoch 88/100
8000/8000 [==============================]

In [13]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1521   74]
 [ 212  193]]


# Initialising the 3rd ANN

In [15]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 95us/step - loss: 0.5206 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4492 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 75us/step - loss: 0.4366 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 75us/step - loss: 0.4309 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 75us/step - loss: 0.4287 - accuracy: 0.7980
Epoch 6/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4279 - accuracy: 0.7991
Epoch 7/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.4276 - accuracy: 0.8004
Epoch 8/100
8000/8000 [==============================] - 1s 75us/step - loss: 0.4274 - accuracy: 0.8015
Epoch 9/100
8000/8000 [==============================] - 1s 75us/step - loss: 0.4270 - accuracy: 0.8020
Epoch 10/100
8000/8000 [==============================] - 1s 76u

8000/8000 [==============================] - 1s 78us/step - loss: 0.3911 - accuracy: 0.8386
Epoch 80/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.3905 - accuracy: 0.8393
Epoch 81/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.3899 - accuracy: 0.8397
Epoch 82/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.3893 - accuracy: 0.8399
Epoch 83/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.3889 - accuracy: 0.8403
Epoch 84/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.3883 - accuracy: 0.8397
Epoch 85/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.3877 - accuracy: 0.8405
Epoch 86/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.3873 - accuracy: 0.8414
Epoch 87/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.3866 - accuracy: 0.8409
Epoch 88/100
8000/8000 [==============================] - 1s 78us/st

In [16]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1530   65]
 [ 257  148]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [18]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print("accuracies:%s\nmean:%s\nvariance:%s".format(accuracies, mean, variance))

accuracies:%s
mean:%s
variance:%s


# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [19]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

# Tuning the ANN

In [20]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [22]:
parameters = {'batch_size': [10, 24, 32],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/50
7200/7200 [==============================] - 1s 114us/step - loss: 0.4909 - accuracy: 0.7968
Epoch 2/50
7200/7200 [==============================] - 1s 88us/step - loss: 0.4348 - accuracy: 0.7971
Epoch 3/50
7200/7200 [==============================] - 1s 89us/step - loss: 0.4340 - accuracy: 0.7971
Epoch 4/50
7200/7200 [==============================] - 1s 88us/step - loss: 0.4308 - accuracy: 0.7971
Epoch 5/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.4244 - accuracy: 0.7971
Epoch 6/50
7200/7200 [==============================] - 1s 89us/step - loss: 0.4256 - accuracy: 0.8165
Epoch 7/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.4250 - accuracy: 0.8229
Epoch 8/50
7200/7200 [==============================] - 1s 92us/step - loss: 0.4248 - accuracy: 0.8253
Epoch 9/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.4257 - accuracy: 0.8257
Epoch 10/50
7200/7200 [==============================] - 1s 90us/step - 

KeyboardInterrupt: 

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)